# Parse data structure into a pandas DataFrame
Parse data paths from the Smart-Kages folder structure and store them in a pandas DataFrames.

Also, load time adjustments to help estimate start and end times for each 1-hour segment.

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import sleap_io as sio

from smart_kages_movement.datetime import find_segment_overlaps
from smart_kages_movement.io import (
    adjust_start_datetimes,
    parse_data_into_df,
)

## Configuration
Define some global variables and paths.

In [3]:
FPS = 2  # Frames per second for all videos

# Path to the data directory containing all kages.
data_dir = Path.home() / "Data" / "Smart-Kages"
assert data_dir.exists(), f"Data directory {data_dir} does not exist."

# Path for saving the processed data (in this case, a subfolder of 'data_dir').
save_dir = data_dir / "movement_analysis"
save_dir.mkdir(parents=True, exist_ok=True)

## Aggregate segment paths into a single dataframe

The data is stored per Smart-Kage, in folders names as `kageN`, e.g. `kage1`, `kage2`, etc.

Each Smart-Kage folder contains:
- daily videos are stored in `videos/YYYY/MM/DD/`, split into 1-hour segments. Each 1-hour segment is an `.mp4` file named `kageN_YYYYMMDD_HHMMSS.mp4`.
- corresponding DeepLabCut (DLC) predictions are stored in `analysis/dlc_output/YYYY/MM/DD/`. Each 1-hour `.h5` file therein is prefixed with `kageN_YYYYMMDD_HHMMSS`.

Let's parse the relevant parts of the data structure into a single dataframe.

In [4]:
df = parse_data_into_df(data_dir)
df.head()

Found 2 kage directories:  kage1 kage3
Found a total of 1615 .h5 pose files output by DLC.


start_datetime  \
kage  date     hour                       
kage1 20240403 09   2024-04-03 09:54:20   
               10   2024-04-03 10:00:02   
               11   2024-04-03 11:01:03   
               12   2024-04-03 12:01:04   
               13   2024-04-03 13:01:03   

                                                        pose_file_path  \
kage  date     hour                                                      
kage1 20240403 09    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               10    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               12    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               13    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   

                                                       video_file_path  
kage  date     hour                                                     
kage1 20240403 09    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               10    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               12    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               13    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...

## Add some video metadata
We reach each video's `n_frames`, `height`, `width`, and `n_channels` from the video file itself, using `sleap-io`.

These metadata are added as columns to the dataframe.

In [5]:
video_shapes = pd.DataFrame(
    np.zeros((len(df), 4), dtype=int),
    index=df.index,
    columns=["n_frames", "height", "width", "n_channels"],
)

for idx, row in df.iterrows():
    video_path = row["video_file_path"]
    video = sio.load_video(video_path)  # Lazy-Load the video using sleap_io
    # Extract video shape information
    video_shapes.loc[idx, "n_frames"] = video.shape[0]
    video_shapes.loc[idx, "height"] = video.shape[1]
    video_shapes.loc[idx, "width"] = video.shape[2]
    video_shapes.loc[idx, "n_channels"] = (
        video.shape[3] if len(video.shape) > 3 else 1
    )
    video.close()  # Close the video to free resources

# Concatenate the video shapes with the original DataFrame
df = pd.concat([df, video_shapes], axis=1)

Let's see if there are any videos where `n_channels` is not 3, which would indicate a non-RGB video.

In [6]:
df_non_rgb = df[df["n_channels"] != 3]
print(f"Founcd {len(df_non_rgb)} non-RGB videos.")
df_non_rgb.head(len(df_non_rgb))

Founcd 7 non-RGB videos.


start_datetime  \
kage  date     hour                       
kage1 20240403 11   2024-04-03 11:01:03   
      20240404 04   2024-04-04 04:01:04   
               06   2024-04-04 06:01:03   
kage3 20240411 02   2024-04-11 02:00:02   
      20240415 14   2024-04-15 14:00:57   
      20240418 18   2024-04-18 18:01:03   
      20240501 15   2024-05-01 15:04:45   

                                                        pose_file_path  \
kage  date     hour                                                      
kage1 20240403 11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
      20240404 04    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               06    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
kage3 20240411 02    /Users/nsirmpilatze/Data/Smart-Kages/kage3/ana...   
      20240415 14    /Users/nsirmpilatze/Data/Smart-Kages/kage3/ana...   
      20240418 18    /Users/nsirmpilatze/Data/Smart-Kages/kage3/ana...   
      20240501 15    /Users/nsirmpilatze/Data/Smart-Kages/kage3/ana...   

                                                       video_file_path  \
kage  date     hour                                                      
kage1 20240403 11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   
      20240404 04    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   
               06    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   
kage3 20240411 02    /Users/nsirmpilatze/Data/Smart-Kages/kage3/vid...   
      20240415 14    /Users/nsirmpilatze/Data/Smart-Kages/kage3/vid...   
      20240418 18    /Users/nsirmpilatze/Data/Smart-Kages/kage3/vid...   
      20240501 15    /Users/nsirmpilatze/Data/Smart-Kages/kage3/vid...   

                     n_frames  height  width  n_channels  
kage  date     hour                                       
kage1 20240403 11        7064     376    500           1  
      20240404 04        7023     376    500           1  
               06        7019     376    500           1  
kage3 20240411 02        7163     376    500           1  
      20240415 14        7066     376    500           1  
      20240418 18        7046     376    500           1  
      20240501 15        6613     376    500           1

## Adjust datetimes for each segment
Above, we derived the start datetime of each video from its filename.
However, this is usually not the exact time when the video started recording.

We can find the exact start time for each video using the `adjustments.txt` file.
There is one such file per day, stored in `kageN/videos/YYYY/MM/DD/`.

`adjustments.txt` contains one row per video file, formatted as `video_file:H,M,S`.
For example, this could be `kage1_20240420_000002.mp4:0,0,6`.

The hours, minutes, and seconds represent the offset relative to time `00:00:00` of that day.
Negative values are possible, e.g. `kage3_20240425_070002.mp4:7,-1,21`
*probably* means the video started at `06:59:21` on that day.

In [7]:
df = adjust_start_datetimes(df)

# For each kage, the start_datetime values should be monotonic increasing
for kage in df.index.get_level_values("kage").unique():
    kage_start_datetimes = df.loc[kage, "start_datetime"].values
    assert np.all(np.diff(kage_start_datetimes) >= pd.Timedelta(0)), (
        f"Start datetimes for kage {kage} are not monotonic increasing."
    )

df.head()

start_datetime  \
kage  date     hour                       
kage1 20240403 09   2024-04-03 09:54:24   
               10   2024-04-03 10:00:06   
               11   2024-04-03 11:01:07   
               12   2024-04-03 12:01:08   
               13   2024-04-03 13:01:07   

                                                        pose_file_path  \
kage  date     hour                                                      
kage1 20240403 09    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               10    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               12    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               13    /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   

                                                       video_file_path  \
kage  date     hour                                                      
kage1 20240403 09    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   
               10    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   
               11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   
               12    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   
               13    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...   

                     n_frames  height  width  n_channels  
kage  date     hour                                       
kage1 20240403 09         665     376    500           3  
               10        7183     376    500           3  
               11        7064     376    500           1  
               12        7059     376    500           3  
               13        7058     376    500           3

Based on the `n_frames` and the `fps` we can estimate each segment's `duration`.

Adding the `duration` to the `start_datetime`, we can get an estimate for `end_datetime`.

In [8]:
FPS = 2  # frames per second
df["duration"] = df["n_frames"] * pd.Timedelta(1 / FPS, "sec")

# Ensure that no segment lasts longer than an hour
assert df["duration"].max() <= pd.Timedelta(1, "hour"), (
    "Some segments last longer than an hour, which is unexpected."
)
# Calculate end datetime based on start datetime and duration
df["end_datetime"] = df["start_datetime"] + df["duration"]

# Reorder columns for readability
df = df[
    [
        "start_datetime",
        "end_datetime",
        "duration",
        "n_frames",
        "n_channels",
        "height",
        "width",
        "pose_file_path",
        "video_file_path",
    ]
]
df.head()

start_datetime            end_datetime  \
kage  date     hour                                               
kage1 20240403 09   2024-04-03 09:54:24 2024-04-03 09:59:56.500   
               10   2024-04-03 10:00:06 2024-04-03 10:59:57.500   
               11   2024-04-03 11:01:07 2024-04-03 11:59:59.000   
               12   2024-04-03 12:01:08 2024-04-03 12:59:57.500   
               13   2024-04-03 13:01:07 2024-04-03 13:59:56.000   

                                  duration  n_frames  n_channels  height  \
kage  date     hour                                                        
kage1 20240403 09   0 days 00:05:32.500000       665           3     376   
               10   0 days 00:59:51.500000      7183           3     376   
               11          0 days 00:58:52      7064           1     376   
               12   0 days 00:58:49.500000      7059           3     376   
               13          0 days 00:58:49      7058           3     376   

                     width                                     pose_file_path  \
kage  date     hour                                                             
kage1 20240403 09      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               10      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               11      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               12      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               13      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   

                                                       video_file_path  
kage  date     hour                                                     
kage1 20240403 09    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               10    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               12    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               13    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...

Let's find out if there are any overlapping segments based on the estimated `start_datetime` and `end_datetime`.

In [9]:
overlaps = find_segment_overlaps(df)
overlaps

Found 10 overlapping segments.


,segment_A,segment_B,end_A,start_B,overlap_duration
0,"(kage1, 20240426, 07)","(kage1, 20240426, 08)",2024-04-26 08:05:00.500,2024-04-26 08:00:06,00:04:54.500000
1,"(kage1, 20240427, 07)","(kage1, 20240427, 08)",2024-04-27 08:06:00.000,2024-04-27 08:00:06,00:05:54
2,"(kage1, 20240428, 07)","(kage1, 20240428, 08)",2024-04-28 08:06:00.500,2024-04-28 08:00:06,00:05:54.500000
3,"(kage1, 20240429, 07)","(kage1, 20240429, 08)",2024-04-29 08:06:00.000,2024-04-29 08:00:06,00:05:54
4,"(kage1, 20240430, 07)","(kage1, 20240430, 08)",2024-04-30 08:05:00.000,2024-04-30 08:00:06,00:04:54
5,"(kage1, 20240502, 07)","(kage1, 20240502, 08)",2024-05-02 08:07:00.000,2024-05-02 08:00:06,00:06:54
6,"(kage1, 20240505, 07)","(kage1, 20240505, 08)",2024-05-05 08:09:00.500,2024-05-05 08:00:06,00:08:54.500000
7,"(kage1, 20240507, 07)","(kage1, 20240507, 08)",2024-05-07 08:08:00.000,2024-05-07 08:00:06,00:07:54
8,"(kage3, 20240425, 06)","(kage3, 20240425, 07)",2024-04-25 06:59:52.500,2024-04-25 06:59:21,00:00:31.500000
9,"(kage3, 20240504, 06)","(kage3, 20240504, 07)",2024-05-04 06:59:39.000,2024-05-04 06:59:21,00:00:18


## Save the dataframes as CSV files
Finally, we save both the `df` and the `overlaps` dataframes to CSV files for later use.

In [10]:
df_file_path = save_dir / "all_segments.csv"
overlaps_file_path = save_dir / "segment_overlaps.csv"
df.to_csv(df_file_path)
overlaps.to_csv(overlaps_file_path)
print(f"Dataframe saved to {df_file_path}")
print(f"Overlaps saved to {overlaps_file_path}")

Dataframe saved to /Users/nsirmpilatze/Data/Smart-Kages/movement_analysis/all_segments.csv
Overlaps saved to /Users/nsirmpilatze/Data/Smart-Kages/movement_analysis/segment_overlaps.csv
